In [1]:
import sys
import os.path
sys.path.append(os.path.join('..', 'util'))
import set_compiler
set_compiler.install()
import pyximport
pyximport.install()
import numpy as np
import pylab
import filtering
from timer import Timer
import threading
import time

Compiling with /usr/local/bin/clang-omp


In [2]:
def iplot(image):
    
    pylab.gray()

    pylab.imshow(image[1200:1800, 3000:3500])
    pylab.title(str(image))
    pylab.show()
    
    
def numpy_median(image, iterations=10):
    ''' filter using numpy '''
    for i in range(iterations):
        padded = np.pad(image, 1, mode='edge')
        stacked = np.dstack((padded[:-2,  :-2], padded[:-2,  1:-1], padded[:-2,  2:],
                             padded[1:-1, :-2], padded[1:-1, 1:-1], padded[1:-1, 2:],
                             padded[2:,   :-2], padded[2:,   1:-1], padded[2:,   2:]))
        image = np.median(stacked, axis=2)

    return image

input_image = np.load('image.npz')['image'].astype(np.float32)
from_numpy1 = numpy_median(input_image, 1)#doesnt work
from_numpy2 = numpy_median(input_image, 2)
from_numpy5 = numpy_median(input_image, 5)#doesnt work
from_numpy7 = numpy_median(input_image, 7)
from_numpy9 = numpy_median(input_image, 9)
from_numpy10 = numpy_median(input_image, 10)

In [109]:
1%2

1

In [110]:
def py_median_3x3_threading(image, iterations, num_threads):
    
    ''' 
    
    repeatedly filter with a 3x3 median using threading
    
    '''
    
    tmpA = image.copy()
    tmpB = np.empty_like(tmpA)
    
    event = np.array([[threading.Event()]*iterations]*num_threads)
    
    for n in range(num_threads):
        thread = threading.Thread(target=worker, args=(tmpA, tmpB, iterations, n, 
                                                       num_threads, event))
        
        thread.start()
    thread.join()
    
    
    if  iterations % 2 == 1:
        return tmpB
    else:
        return tmpA

In [120]:
def worker(in_coord, out_coord, iterations, threadid, thread_num, event_array):
    for i in range(iterations):

        if thread_num == 1:
            pass
        elif i == 0:
            pass
        elif threadid == 0:
            event_array[threadid + 1,i - 1].wait()
        elif threadid == thread_num - 1:
            event_array[threadid - 1, i - 1].wait()
        else:
            event_array[threadid - 1, i - 1].wait()#causes to current thread
            event_array[threadid + 1, i - 1].wait()# to wait on these threads 
                                                   #to be complete
                
        filtering.median_3x3(in_coord, out_coord, threadid, thread_num)
        event_array[threadid, i].set() #tell the event that this thread is complete
        in_coord, out_coord = out_coord, in_coord

In [129]:
from_cython = py_median_3x3_threading(input_image, 1,5)
assert np.all(from_cython == from_numpy1)

In [61]:
def worker2(in_coord, out_coord, iterations, threadid, thread_num, event_array):
    for i in range(iterations):
        for j in range(0, len(event_array), thread_num):
        
            if thread_num > 1:

                if i == 0:
                    pass
                elif threadid == 0:
                    event_array[j + 1, threadid + 1, i - 1].wait()
                    event_array[j + 1, threadid + 1, i].wait()
                elif threadid == thread_num - 1:
                    event_array[j - 1, threadid - 1, i - 1].wait()
                    event_array[j - 1, threadid - 1, i].wait()
                else:
                    event_array[j - 1, threadid - 1, i - 1].wait()
                    event_array[j - 1, threadid - 1, i - 1].wait()
                    event_array[j - 1, threadid - 1, i - 1].wait()

                        
                    event_array[j, threadid - 1, i - 1].wait()
                    event_array[j, threadid - 1, i - 1].wait()
                    event_array[j, threadid - 1, i - 1].wait()

            filtering.median_3x3(in_coord, out_coord, threadid + j, 1)
            event_array[threadid + j, threadid, i].set() #tell the event that this thread is complete
            in_coord, out_coord = out_coord, in_coord

IndentationError: expected an indented block (<ipython-input-61-daf530aba385>, line 5)